## 1. Package Preparation

In [1]:
# clear warnings
import warnings
warnings.simplefilter("ignore")

In [2]:
#import packages

# import keras
from keras import backend as K
K.set_image_dim_ordering('tf') # note that we need to have tensorflow dimension ordering still because of the weigths.
print('The backend is:',K.backend())
import tensorflow as tf
print(K.image_dim_ordering()) # should say tf
print(tf.__version__) # tested for 1.11.0
import cv2
import keras
print(keras.__version__) # tested for 2.2.4

Using TensorFlow backend.


The backend is: tensorflow
tf
1.9.0
2.2.2


In [3]:
# Import relevant packages
from __future__ import absolute_import, division, print_function # make it compatible w Python 2
import os
import h5py # to handle weights
import os, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

# to read image
from PIL import Image

# relative keras packages
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation, ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras.preprocessing import image

# useful packages from sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from IPython.display import Image, display

# fix random seed for reproducibility
np.random.seed(150)

## 2. Read data from preprocessed pickle file

In [4]:
# We have preprocessed our data and saved in a pickle file
# Now we only need to read this pickle file
map_data = pd.read_pickle('position_df.pkl')

In [ ]:
map_data.head()

The column 'index' in the map_data above is used to help match the location data for the 'location' column.

We only need the data in the 'label' column and 'image' column to train our model, so we drop those two unnecessary columns.

In [5]:
# Resize from 640 * 640 * 3

map_data['image'] = map_data['image'].apply(lambda x: cv2.resize(x,(200,125)))

In [6]:
map_data.drop(['location','index'],axis=1,inplace=True)

In [7]:
# Show the head of our map_data
map_data.head()

,label,image
0,3,"[[[85, 59, 55], [72, 44, 40], [84, 57, 53], [9..."
1,4,"[[[66, 42, 42], [70, 37, 37], [72, 40, 40], [6..."
2,4,"[[[66, 52, 79], [78, 62, 77], [81, 68, 80], [8..."
3,4,"[[[111, 96, 104], [100, 85, 92], [159, 151, 15..."
4,4,"[[[202, 195, 196], [203, 198, 198], [95, 83, 9..."


## 3. Split data for cross validation

In [8]:
# Read X and y data

Xs = np.array(map_data.iloc[:,1].tolist())
ys = np.array(map_data.iloc[:,0])

In [9]:
# Split data into training data (including training and validation) and test data
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size = 0.2, random_state = 0)

# Split training data into true training data and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [10]:
# confirm the shape and type of our data is right
print('The length of X_train is', len(X_train))

print('The length of X_val is', len(X_val))

print('The length of X_test is', len(X_test))

print('The length of y_train is', len(y_train))

print('The length of y_val is', len(y_val))

print('The length of y_test is', len(y_test))

The length of X_train is 2235
The length of X_val is 559
The length of X_test is 699
The length of y_train is 2235
The length of y_val is 559
The length of y_test is 699


In [11]:
# A small check of total data amount before training the model
check_X = (len(X_train) + len(X_val) + len(X_test)) == len(map_data)
check_y = (len(y_train) + len(y_val) + len(y_test)) == len(map_data)

print('X data sets equality is:', check_X)
print('y data sets equality is:', check_y)

X data sets equality is: True
y data sets equality is: True


## 4. One-hot encoding and normalizing our input data

In [12]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [13]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(y_test)

## 5. Model Training

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [19]:
# First type of CNN model
input_size=(125,200,3)
num_classes=5

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 125, 200, 32)      896       
_________________________________________________________________
dropout_4 (Dropout)          (None, 125, 200, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 100, 32)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 198400)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               101581312 
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
__________

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [17]:
# Another type of CNN model
input_size=(125,200,3)
num_classes=5

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 125, 200, 32)      896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 125, 200, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 100, 32)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 198400)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 992005    
Total params: 992,901
Trainable params: 992,901
Non-trainable params: 0
_________________________________________________________________
None
CNN Model created.


In [20]:
X_train.shape

(2235, 125, 200, 3)

In [21]:
#batch size，learning rate can be modified before training

batch_size=60
seed = 7
np.random.seed(seed)
model.fit(X_train, y_train, batch_size = batch_size, nb_epoch = epochs)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=60)

Epoch 1/3
2235/2235 [==============================] - 2353s 1s/step - loss: 1.6213 - acc: 0.2653
Epoch 2/3
2235/2235 [==============================] - 1815s 812ms/step - loss: 1.4533 - acc: 0.3588
Epoch 3/3
2235/2235 [==============================] - 2052s 918ms/step - loss: 1.3322 - acc: 0.4157


In [22]:
# Final evaluation of the model on test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

print("done")

Accuracy: 42.78%
done


In [24]:
# Final evaluation of the model on test
scores = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: %.2f%%" % (scores[1]*100))

print("done")

Training Accuracy: 52.84%
done


In [25]:
# Final evaluation of the model on test
scores = model.evaluate(X_val, y_val, verbose=0)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))

print("done")

Validation Accuracy: 43.47%
done


In [26]:
y_pred=model.predict(X_test)
print("y_pred is ",y_pred)

y_pred is  [[0.16616884 0.1834371  0.28311354 0.24392208 0.12335841]
 [0.36161795 0.20711294 0.23451313 0.1337956  0.0629604 ]
 [0.282091   0.33298552 0.14026824 0.10466771 0.13998757]
 ...
 [0.594185   0.2859091  0.0856473  0.02471039 0.00954829]
 [0.12159665 0.09033503 0.42159232 0.30151638 0.06495959]
 [0.45229322 0.3620299  0.08876586 0.05765385 0.03925715]]


In [27]:
y_test

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [28]:
##store the weights
for layer in model.layers:
    weights = layer.get_weights() 

In [29]:
weights

[array([[ 0.08077971, -0.08494316,  0.10738301, -0.02133705,  0.00033168],
        [-0.03683984, -0.08718871, -0.08432052,  0.08939377, -0.01828957],
        [-0.01485827,  0.01543096,  0.05174096,  0.08669905,  0.01566698],
        ...,
        [ 0.00121219, -0.05390524, -0.00892309, -0.0747997 , -0.03409497],
        [-0.0796201 , -0.08361286, -0.07777672, -0.07893319, -0.04651123],
        [ 0.01004251,  0.05703814,  0.00833634, -0.00343164, -0.03502527]],
       dtype=float32),
 array([-0.02979133,  0.01614214,  0.0238754 , -0.00688321, -0.00334301],
       dtype=float32)]

In [30]:
len(weights)

2

In [31]:
## store the model
model.save('cnn_trial_with_2235.h5')

In [ ]:
###read the stored the model
import h5py

def print_keras_wegiths(weight_file_path):
    # Read weights_h5 file and give a 'file'
    
    f = h5py.File(weight_file_path)  
    
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))  
            # Output: attrs information stored in the file
            # Generally speaking, it's the names of each layer

        for layer, g in f.items():  
            # Read names of each layer and the arrs group containing information of each layer
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items(): 
                print("      {}: {}".format(key, value))  

            print("    Dataset:")
            for name, d in g.items():
                print("      {}: {}".format(name, d.value.shape))
                print("      {}: {}".format(name. d.value))
    finally:
        f.close()